# Assignment #2: Language models

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment
* Optionally read a short article on the importance of corpora


## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile
from collections import defaultdict
from functools import reduce
import pandas as pd

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
# You may have to adjust the path
corpus = open('./Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [3]:
pattern = 'Nils Holgersson'
width = 25

In [4]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [5]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [6]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [7]:
len(set(tokenize(clean_corpus)))

44256

Lowercased text

In [8]:
len(set(tokenize(clean_corpus.lower())))

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [9]:
nonletter = '[^\p{L}.;:?!]+'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [10]:
def clean(string):
    return re.sub(nonletter, ' ', string)

In [11]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [12]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

In [13]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [14]:
sentence_boundaries = '[.?!]\p{Zs}+(?<cap>\p{Lu})'

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [15]:
sentence_markup = ' </s>\n<s> \g<cap>'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [16]:
text = re.sub(sentence_boundaries, sentence_markup, test_para)

In [17]:
print(text)

En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.


In [18]:
print(text)

En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.


The output should look like this:

`En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.`

Insert markup codes in the beginning and end of the text

In [19]:
text = ' <s> ' + text + ' </s> '

In [20]:
print(text)

 <s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden. </s> 


The output should look like this:

`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden. </s>`

Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [21]:
text = re.sub('[.;:?!\p{Zs}]+', ' ', text)

In [22]:
print(text)

 <s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden </s> 


The output should look like this:
    
`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden </s>`

Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [23]:
def segment_sentences(text):
    sentence_boundaries = '[.;:?!]\p{Zs}+(?<cap>\p{Lu})'
    sentence_markup = ' </s>\n<s> \g<cap>'
    
    clean_text = re.sub(sentence_boundaries, sentence_markup, text)
    clean_text = ' <s> ' + clean_text + ' </s>'
    clean_text = re.sub('[.;:?!\p{Zs}]+', ' ', clean_text)
    clean_text = clean_text.lower()
    
    return clean_text

In [24]:
print(segment_sentences(test_para))

 <s> en gång hade de på mårbacka en barnpiga, som hette back-kajsa </s>
<s> hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

In [25]:
lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Massa tincidunt nunc pulvinar sapien. Metus vulputate eu scelerisque felis imperdiet proin fermentum. Porttitor lacus luctus accumsan tortor posuere ac ut consequat semper. Massa enim nec dui nunc mattis. Aenean et tortor at risus viverra adipiscing at in. Sem fringilla ut morbi tincidunt. Orci dapibus ultrices in iaculis nunc sed augue lacus viverra. Donec ac odio tempor orci. Ipsum dolor sit amet consectetur adipiscing elit. Cras semper auctor neque vitae tempus quam pellentesque nec nam. Vitae auctor eu augue ut lectus arcu bibendum at. Augue eget arcu dictum varius duis at consectetur lorem. Sodales ut eu sem integer vitae justo. Vestibulum lorem sed risus ultricies tristique nulla. Senectus et netus et malesuada. Amet purus gravida quis blandit turpis cursus in hac habitasse. Pharetra diam sit amet nisl suscipit adipiscing bibendum est. Metus dictum at tempor commodo ullamcorper a lacus vestibulum sed. Lectus mauris ultrices eros in cursus turpis massa. Quis enim lobortis scelerisque fermentum dui. Ante in nibh mauris cursus mattis. Adipiscing elit ut aliquam purus sit. Suscipit tellus mauris a diam maecenas sed enim ut sem. Arcu felis bibendum ut tristique et. Neque aliquam vestibulum morbi blandit cursus risus at. Sed turpis tincidunt id aliquet risus feugiat in ante. Facilisis volutpat est velit egestas dui id ornare arcu odio. Consectetur a erat nam at lectus. Cum sociis natoque penatibus et magnis dis. Blandit turpis cursus in hac habitasse platea dictumst. Platea dictumst quisque sagittis purus sit amet volutpat consequat mauris. Faucibus vitae aliquet nec ullamcorper sit amet risus nullam eget. Aliquet lectus proin nibh nisl condimentum id venenatis a condimentum. Eget mauris pharetra et ultrices neque ornare aenean euismod elementum. Aliquam malesuada bibendum arcu vitae elementum curabitur. Enim nec dui nunc mattis enim ut tellus. Neque laoreet suspendisse interdum consectetur libero id. Ac placerat vestibulum lectus mauris ultrices eros. Tellus mauris a diam maecenas sed enim ut. Egestas sed tempus urna et pharetra pharetra massa. Arcu dictum varius duis at consectetur lorem. Bibendum est ultricies integer quis auctor. Praesent elementum facilisis leo vel fringilla. Pretium nibh ipsum consequat nisl vel pretium lectus quam. Malesuada bibendum arcu vitae elementum curabitur vitae nunc sed. Eleifend quam adipiscing vitae proin sagittis. Ipsum nunc aliquet bibendum enim facilisis gravida neque. Dignissim convallis aenean et tortor at risus viverra adipiscing. Aenean euismod elementum nisi quis eleifend quam adipiscing. Vitae tortor condimentum lacinia quis. Praesent elementum facilisis leo vel fringilla est ullamcorper eget nulla. Volutpat est velit egestas dui id ornare arcu odio ut. Pretium fusce id velit ut. Nunc sed blandit libero volutpat sed cras ornare arcu. Placerat vestibulum lectus mauris ultrices eros. Tortor vitae purus faucibus ornare suspendisse sed nisi lacus. Turpis nunc eget lorem dolor. Tincidunt lobortis feugiat vivamus at augue eget. Feugiat vivamus at augue eget. Purus gravida quis blandit turpis cursus in hac habitasse platea. Ut porttitor leo a diam sollicitudin tempor. Vitae et leo duis ut diam quam. Non arcu risus quis varius quam. Vulputate ut pharetra sit amet aliquam id diam. Sit amet massa vitae tortor condimentum lacinia quis vel eros. At augue eget arcu dictum varius. Elementum curabitur vitae nunc sed. Sit amet dictum sit amet justo donec enim diam vulputate. Vel pharetra vel turpis nunc eget lorem dolor sed viverra. Elementum eu facilisis sed odio morbi quis commodo odio aenean. Et tortor consequat id porta nibh venenatis cras sed felis. Imperdiet nulla malesuada pellentesque elit eget gravida cum sociis natoque. Commodo viverra maecenas accumsan lacus vel facilisis. Tempor orci eu lobortis elementum nibh tellus. Ut ornare lectus sit amet est placerat in egestas erat. Leo urna molestie at elementum eu facilisis. Viverra tellus in hac habitasse. Facilisis magna etiam tempor orci eu. Amet nulla facilisi morbi tempus iaculis urna id volutpat. Enim blandit volutpat maecenas volutpat blandit aliquam etiam. In hendrerit gravida rutrum quisque. Et magnis dis parturient montes nascetur ridiculus. In fermentum posuere urna nec. Nulla aliquet enim tortor at auctor urna. Diam vulputate ut pharetra sit amet. At varius vel pharetra vel turpis nunc eget lorem. Cursus eget nunc scelerisque viverra mauris in aliquam sem. Risus in hendrerit gravida rutrum quisque non tellus orci ac. Elit ut aliquam purus sit amet luctus venenatis. Blandit volutpat maecenas volutpat blandit aliquam etiam. Nec feugiat nisl pretium fusce id velit ut tortor. Eu tincidunt tortor aliquam nulla facilisi cras fermentum odio eu. Integer enim neque volutpat ac tincidunt. Dictumst quisque sagittis purus sit amet volutpat consequat. Vitae tortor condimentum lacinia quis vel eros. Ullamcorper sit amet risus nullam eget felis eget. Nunc id cursus metus aliquam eleifend mi. Neque gravida in fermentum et sollicitudin ac orci phasellus egestas. Vitae tortor condimentum lacinia quis vel eros donec ac. Vel elit scelerisque mauris pellentesque. Aenean sed adipiscing diam donec adipiscing tristique. Vitae ultricies leo integer malesuada nunc vel risus commodo. At consectetur lorem donec massa sapien faucibus. Porta non pulvinar neque laoreet suspendisse interdum consectetur libero. Sit amet consectetur adipiscing elit. Id consectetur purus ut faucibus pulvinar elementum integer enim neque. Tellus id interdum velit laoreet id donec. Sed nisi lacus sed viverra tellus in hac habitasse. Gravida arcu ac tortor dignissim. Ac turpis egestas maecenas pharetra convallis posuere morbi leo. Adipiscing diam donec adipiscing tristique. Lectus quam id leo in vitae. Elit scelerisque mauris pellentesque pulvinar pellentesque. Est ultricies integer quis auctor. Tellus id interdum velit laoreet id donec. Nullam eget felis eget nunc lobortis mattis aliquam faucibus. Curabitur gravida arcu ac tortor dignissim convallis aenean et tortor. Tincidunt praesent semper feugiat nibh sed pulvinar proin. Bibendum est ultricies integer quis auctor. Cursus eget nunc scelerisque viverra mauris. Sit amet purus gravida quis blandit. Vel eros donec ac odio tempor orci dapibus. Amet justo donec enim diam. Lectus quam id leo in vitae. Suspendisse interdum consectetur libero id faucibus nisl. Neque volutpat ac tincidunt vitae. Eget dolor morbi non arcu risus quis varius quam quisque. Posuere sollicitudin aliquam ultrices sagittis orci a scelerisque purus semper. Gravida dictum fusce ut placerat orci nulla. Lorem sed risus ultricies tristique nulla aliquet enim tortor at. Vulputate odio ut enim blandit. Tempus egestas sed sed risus pretium quam vulputate. Turpis egestas integer eget aliquet nibh praesent. Id semper risus in hendrerit gravida rutrum. Tempus urna et pharetra pharetra massa massa ultricies mi. Urna cursus eget nunc scelerisque viverra mauris in aliquam. Elit scelerisque mauris pellentesque pulvinar pellentesque. Vestibulum mattis ullamcorper velit sed ullamcorper morbi tincidunt ornare. Varius sit amet mattis vulputate enim nulla aliquet. Vel quam elementum pulvinar etiam non quam lacus. Est sit amet facilisis magna etiam tempor. Aliquam vestibulum morbi blandit cursus risus. Euismod nisi porta lorem mollis aliquam ut porttitor. Nunc scelerisque viverra mauris in aliquam. Phasellus vestibulum lorem sed risus ultricies. Feugiat scelerisque varius morbi enim nunc faucibus a pellentesque. Amet aliquam id diam maecenas ultricies mi eget mauris pharetra. Tellus rutrum tellus pellentesque eu tincidunt tortor aliquam nulla facilisi. Sed id semper risus in hendrerit gravida. Placerat vestibulum lectus mauris ultrices. Ultrices sagittis orci a scelerisque purus semper. Dignissim diam quis enim lobortis scelerisque fermentum. Lorem ipsum dolor sit amet consectetur. Odio pellentesque diam volutpat commodo sed. Sed augue lacus viverra vitae congue. Suspendisse faucibus interdum posuere lorem ipsum. Augue interdum velit euismod in pellentesque massa. Diam quis enim lobortis scelerisque. Sit amet consectetur adipiscing elit pellentesque habitant morbi. Accumsan in nisl nisi scelerisque eu. Odio tempor orci dapibus ultrices in iaculis nunc sed augue. At quis risus sed vulputate. Et molestie ac feugiat sed lectus vestibulum mattis ullamcorper velit. Facilisis gravida neque convallis a cras semper. Nulla posuere sollicitudin aliquam ultrices sagittis orci. In nulla posuere sollicitudin aliquam ultrices sagittis. Feugiat in ante metus dictum at tempor commodo ullamcorper. Pulvinar mattis nunc sed blandit libero volutpat. Pellentesque elit ullamcorper dignissim cras tincidunt lobortis. Ullamcorper malesuada proin libero nunc. Vitae proin sagittis nisl rhoncus mattis rhoncus urna neque. Amet purus gravida quis blandit. Blandit massa enim nec dui nunc mattis enim ut tellus. Lorem ipsum dolor sit amet consectetur adipiscing elit ut aliquam. Feugiat scelerisque varius morbi enim nunc. Sagittis id consectetur purus ut faucibus pulvinar. Pretium aenean pharetra magna ac placerat vestibulum lectus mauris ultrices. Mattis vulputate enim nulla aliquet porttitor lacus luctus accumsan. Sit amet consectetur adipiscing elit. Facilisi nullam vehicula ipsum a arcu cursus vitae. Nunc id cursus metus aliquam eleifend. Sed viverra ipsum nunc aliquet bibendum enim. Suspendisse sed nisi lacus sed viverra tellus in hac habitasse. Congue nisi vitae suscipit tellus. Amet nisl suscipit adipiscing bibendum est ultricies integer quis. Ullamcorper morbi tincidunt ornare massa eget egestas purus viverra. Id aliquet risus feugiat in ante metus dictum at tempor. At risus viverra adipiscing at in tellus integer feugiat. Amet cursus sit amet dictum. Consectetur purus ut faucibus pulvinar elementum integer. Donec ultrices tincidunt arcu non sodales. Placerat vestibulum lectus mauris ultrices eros in. Fringilla phasellus faucibus scelerisque eleifend donec pretium vulputate sapien nec. Sit amet nulla facilisi morbi tempus iaculis urna id volutpat. Dictum non consectetur a erat nam at lectus urna duis. Dictum fusce ut placerat orci nulla pellentesque dignissim. Sagittis vitae et leo duis. Porttitor eget dolor morbi non. Ac orci phasellus egestas tellus rutrum tellus. Gravida rutrum quisque non tellus orci ac auctor augue. Amet dictum sit amet justo donec enim diam. Adipiscing tristique risus nec feugiat in. Bibendum est ultricies integer quis auctor elit. Ac orci phasellus egestas tellus rutrum tellus. Id faucibus nisl tincidunt eget nullam non. Adipiscing diam donec adipiscing tristique risus nec feugiat in fermentum. Diam quam nulla porttitor massa id neque aliquam. Id leo in vitae turpis massa sed elementum tempus egestas. Malesuada pellentesque elit eget gravida cum sociis natoque penatibus et. Eget nullam non nisi est sit amet facilisis. Nec feugiat nisl pretium fusce id velit ut tortor. Gravida arcu ac tortor dignissim convallis aenean et tortor at. Duis ultricies lacus sed turpis. Rhoncus aenean vel elit scelerisque mauris pellentesque pulvinar pellentesque habitant. Sit amet venenatis urna cursus eget nunc scelerisque. Tempus egestas sed sed risus pretium quam vulputate dignissim suspendisse. Consectetur lorem donec massa sapien faucibus et molestie ac. Quis risus sed vulputate odio ut enim blandit volutpat maecenas. Neque convallis a cras semper auctor neque. Eget mi proin sed libero. In hac habitasse platea dictumst quisque sagittis purus sit. Eleifend quam adipiscing vitae proin sagittis nisl rhoncus. Tristique senectus et netus et malesuada fames ac turpis. Metus vulputate eu scelerisque felis imperdiet proin fermentum leo vel. Ultrices eros in cursus turpis massa. Sagittis aliquam malesuada bibendum arcu vitae elementum curabitur vitae nunc. Vitae purus faucibus ornare suspendisse sed nisi lacus sed. At erat pellentesque adipiscing commodo elit at imperdiet dui. Dolor sit amet consectetur adipiscing elit. Aliquam sem fringilla ut morbi tincidunt augue interdum. Hendrerit gravida rutrum quisque non. Enim blandit volutpat maecenas volutpat blandit aliquam etiam erat velit. Lacus viverra vitae congue eu. Enim neque volutpat ac tincidunt vitae semper quis lectus. Sit amet consectetur adipiscing elit ut. Massa sapien faucibus et molestie ac feugiat sed. Imperdiet nulla malesuada pellentesque elit eget gravida cum sociis natoque. Curabitur gravida arcu ac tortor dignissim convallis aenean et tortor. Feugiat scelerisque varius morbi enim. Tortor id aliquet lectus proin nibh. Fringilla urna porttitor rhoncus dolor purus non enim praesent elementum. Mattis rhoncus urna neque viverra justo nec ultrices. Volutpat ac tincidunt vitae semper quis lectus nulla. Adipiscing tristique risus nec feugiat in. Nulla pharetra diam sit amet nisl suscipit adipiscing bibendum est. Amet facilisis magna etiam tempor orci. Massa ultricies mi quis hendrerit. Id faucibus nisl tincidunt eget nullam non. Lacinia at quis risus sed vulputate. Quisque non tellus orci ac auctor augue mauris augue neque. Turpis massa sed elementum tempus egestas. Consectetur a erat nam at lectus urna duis convallis. Fusce ut placerat orci nulla pellentesque dignissim enim sit. Sed elementum tempus egestas sed sed risus pretium quam. Pulvinar mattis nunc sed blandit. Vel pharetra vel turpis nunc eget lorem dolor sed. Ac tortor dignissim convallis aenean et tortor at risus viverra. Feugiat pretium nibh ipsum consequat nisl vel. Vitae tortor condimentum lacinia quis vel eros donec ac odio. Gravida in fermentum et sollicitudin ac orci. Pharetra sit amet aliquam id diam maecenas. Scelerisque felis imperdiet proin fermentum leo vel. Id consectetur purus ut faucibus pulvinar elementum integer. Lacus suspendisse faucibus interdum posuere lorem. Condimentum vitae sapien pellentesque habitant morbi tristique. Nibh sed pulvinar proin gravida hendrerit lectus. Arcu non odio euismod lacinia at quis. Non tellus orci ac auctor. Pellentesque id nibh tortor id aliquet lectus proin. Eu sem integer vitae justo eget magna. Aliquam purus sit amet luctus venenatis lectus magna. Felis eget nunc lobortis mattis aliquam faucibus purus in massa. Ultrices dui sapien eget mi proin sed libero enim sed. Habitant morbi tristique senectus et netus et malesuada fames. Nisl rhoncus mattis rhoncus urna neque viverra justo nec ultrices. Velit euismod in pellentesque massa placerat duis. Id neque aliquam vestibulum morbi blandit cursus. Pulvinar etiam non quam lacus suspendisse faucibus interdum. Sed sed risus pretium quam vulputate dignissim suspendisse in est. Feugiat pretium nibh ipsum consequat nisl vel. Iaculis at erat pellentesque adipiscing commodo. Egestas erat imperdiet sed euismod. Dui id ornare arcu odio ut sem nulla pharetra diam. Scelerisque in dictum non consectetur a erat. Id neque aliquam vestibulum morbi. Nunc lobortis mattis aliquam faucibus purus in massa. Cursus sit amet dictum sit. Et malesuada fames ac turpis egestas sed tempus urna. Consectetur adipiscing elit pellentesque habitant morbi tristique senectus. At urna condimentum mattis pellentesque id nibh. Sagittis vitae et leo duis. Vulputate eu scelerisque felis imperdiet proin fermentum leo vel orci. Sollicitudin aliquam ultrices sagittis orci a. Purus semper eget duis at tellus at urna. Id neque aliquam vestibulum morbi blandit cursus risus. Enim facilisis gravida neque convallis a cras semper. Viverra accumsan in nisl nisi scelerisque eu ultrices vitae. Placerat duis ultricies lacus sed. Imperdiet nulla malesuada pellentesque elit eget gravida cum. Lorem sed risus ultricies tristique nulla aliquet enim tortor. Justo nec ultrices dui sapien eget mi proin sed. Integer vitae justo eget magna fermentum iaculis eu non. Elit pellentesque habitant morbi tristique senectus. Lacus suspendisse faucibus interdum posuere. Sociis natoque penatibus et magnis dis parturient montes nascetur ridiculus. Id aliquet risus feugiat in ante metus dictum at. Feugiat sed lectus vestibulum mattis ullamcorper velit sed. Massa vitae tortor condimentum lacinia quis vel eros donec ac. Pretium quam vulputate dignissim suspendisse in est. Mauris commodo quis imperdiet massa tincidunt nunc. Augue ut lectus arcu bibendum. Sit amet mauris commodo quis imperdiet. Risus feugiat in ante metus dictum at tempor. Eu augue ut lectus arcu bibendum at varius. Viverra vitae congue eu consequat. Rhoncus dolor purus non enim. Ut sem viverra aliquet eget sit amet tellus cras adipiscing. Lacus luctus accumsan tortor posuere ac ut consequat semper viverra. Condimentum vitae sapien pellentesque habitant morbi tristique senectus. Massa sed elementum tempus egestas sed. Aliquet risus feugiat in ante metus. Imperdiet proin fermentum leo vel orci porta. Ac ut consequat semper viverra nam libero justo laoreet sit. Tempus imperdiet nulla malesuada pellentesque elit. Erat imperdiet sed euismod nisi porta lorem mollis. Diam sollicitudin tempor id eu nisl nunc mi. Ullamcorper a lacus vestibulum sed arcu non. Risus nec feugiat in fermentum posuere urna. Egestas pretium aenean pharetra magna ac. Dui id ornare arcu odio ut sem. Lacinia at quis risus sed vulputate. Mattis rhoncus urna neque viverra justo nec. In nisl nisi scelerisque eu. Eget magna fermentum iaculis eu non diam phasellus vestibulum. Bibendum neque egestas congue quisque egestas diam in arcu. Vel pharetra vel turpis nunc eget lorem dolor. In metus vulputate eu scelerisque felis imperdiet. Et sollicitudin ac orci phasellus. Consectetur a erat nam at lectus urna duis. Platea dictumst vestibulum rhoncus est pellentesque elit ullamcorper dignissim cras. Tortor vitae purus faucibus ornare suspendisse sed nisi. Ac tincidunt vitae semper quis lectus nulla at. Duis at tellus at urna condimentum. In vitae turpis massa sed elementum tempus egestas sed sed. Facilisi nullam vehicula ipsum a arcu cursus vitae congue mauris. Velit ut tortor pretium viverra suspendisse potenti nullam. Netus et malesuada fames ac turpis egestas sed tempus. Magna eget est lorem ipsum dolor sit amet consectetur. At consectetur lorem donec massa. Sed viverra tellus in hac habitasse platea. Et malesuada fames ac turpis egestas maecenas pharetra. Mollis aliquam ut porttitor leo a diam sollicitudin tempor id. Eu lobortis elementum nibh tellus molestie. Vitae turpis massa sed elementum. Aliquam ut porttitor leo a diam. Placerat orci nulla pellentesque dignissim enim. Felis donec et odio pellentesque diam volutpat. Nisl nisi scelerisque eu ultrices vitae auctor. Ultricies mi eget mauris pharetra et ultrices. Pulvinar elementum integer enim neque. Eu feugiat pretium nibh ipsum consequat nisl vel. Felis eget nunc lobortis mattis aliquam faucibus purus. Turpis nunc eget lorem dolor sed viverra ipsum nunc aliquet. Dignissim enim sit amet venenatis. Ornare massa eget egestas purus viverra. Mattis nunc sed blandit libero volutpat sed cras ornare arcu. Vulputate enim nulla aliquet porttitor lacus luctus. Dignissim convallis aenean et tortor at risus. Consectetur purus ut faucibus pulvinar elementum integer enim neque. Egestas fringilla phasellus faucibus scelerisque eleifend donec. At ultrices mi tempus imperdiet nulla malesuada pellentesque. Est velit egestas dui id ornare arcu odio. Amet tellus cras adipiscing enim eu. Sed augue lacus viverra vitae congue eu consequat. Diam ut venenatis tellus in metus. In tellus integer feugiat scelerisque varius morbi. Ut pharetra sit amet aliquam id diam maecenas ultricies. Id volutpat lacus laoreet non curabitur gravida arcu. Pulvinar mattis nunc sed blandit libero volutpat sed cras ornare. Vestibulum lectus mauris ultrices eros in cursus. Purus in massa tempor nec feugiat nisl pretium fusce. Tellus molestie nunc non blandit. Aliquet enim tortor at auctor. Egestas pretium aenean pharetra magna ac placerat vestibulum lectus mauris. Malesuada pellentesque elit eget gravida cum sociis natoque penatibus. Congue quisque egestas diam in arcu cursus euismod quis viverra. Dictum non consectetur a erat nam at lectus urna. Interdum velit euismod in pellentesque massa placerat duis ultricies lacus. At urna condimentum mattis pellentesque id. Sed tempus urna et pharetra pharetra. Bibendum enim facilisis gravida neque convallis a cras semper auctor. Sed vulputate odio ut enim. Ullamcorper eget nulla facilisi etiam dignissim diam quis enim. Nunc vel risus commodo viverra maecenas accumsan lacus. Commodo viverra maecenas accumsan lacus vel facilisis volutpat. Enim sed faucibus turpis in eu mi bibendum. Vitae congue mauris rhoncus aenean vel elit scelerisque mauris. Ac auctor augue mauris augue neque gravida in. Orci ac auctor augue mauris augue. Morbi blandit cursus risus at ultrices mi. Tortor pretium viverra suspendisse potenti nullam. Lectus quam id leo in. In mollis nunc sed id semper risus. Eget est lorem ipsum dolor sit amet consectetur adipiscing elit. Lacus luctus accumsan tortor posuere. Volutpat blandit aliquam etiam erat velit. Amet luctus venenatis lectus magna fringilla urna porttitor rhoncus. Aliquam eleifend mi in nulla posuere sollicitudin. Semper eget duis at tellus at urna condimentum. Sit amet consectetur adipiscing elit pellentesque habitant morbi tristique senectus. Eget aliquet nibh praesent tristique magna sit amet purus. Sed odio morbi quis commodo odio aenean sed adipiscing. Sit amet nisl suscipit adipiscing bibendum est. Lobortis feugiat vivamus at augue eget arcu dictum varius. Tincidunt praesent semper feugiat nibh sed. Magna etiam tempor orci eu lobortis elementum nibh tellus molestie. Consequat mauris nunc congue nisi vitae suscipit tellus mauris. Hendrerit dolor magna eget est lorem ipsum. Et netus et malesuada fames ac turpis egestas sed. Mi ipsum faucibus vitae aliquet. Natoque penatibus et magnis dis parturient. Varius sit amet mattis vulputate enim nulla. Viverra orci sagittis eu volutpat odio. Risus at ultrices mi tempus imperdiet nulla malesuada. Cursus sit amet dictum sit. Ante in nibh mauris cursus mattis molestie a. Sit amet cursus sit amet dictum sit amet justo donec. Augue mauris augue neque gravida in fermentum et sollicitudin. Feugiat in ante metus dictum at tempor commodo ullamcorper. Diam sollicitudin tempor id eu nisl nunc mi ipsum faucibus. In nibh mauris cursus mattis molestie a iaculis at erat. Egestas erat imperdiet sed euismod nisi porta. Mi quis hendrerit dolor magna eget est lorem ipsum dolor. Sed tempus urna et pharetra. Mus mauris vitae ultricies leo integer malesuada. Ridiculus mus mauris vitae ultricies leo integer malesuada nunc vel. Sed turpis tincidunt id aliquet risus. Enim neque volutpat ac tincidunt. Egestas erat imperdiet sed euismod nisi porta lorem. Ut porttitor leo a diam. Augue mauris augue neque gravida in fermentum. Sit amet nisl suscipit adipiscing. Vitae semper quis lectus nulla at volutpat diam ut. Vitae suscipit tellus mauris a diam maecenas. Ultrices mi tempus imperdiet nulla malesuada pellentesque elit eget gravida. Quis commodo odio aenean sed adipiscing diam. Purus gravida quis blandit turpis cursus in hac habitasse platea. A condimentum vitae sapien pellentesque habitant morbi tristique. Nunc faucibus a pellentesque sit amet. Egestas maecenas pharetra convallis posuere morbi leo urna molestie. Amet justo donec enim diam vulputate ut pharetra. Id diam vel quam elementum pulvinar etiam non quam lacus. Commodo quis imperdiet massa tincidunt nunc pulvinar sapien et ligula. Leo vel fringilla est ullamcorper eget. Egestas pretium aenean pharetra magna. Rhoncus urna neque viverra justo. Ultricies mi eget mauris pharetra et ultrices neque ornare. Pellentesque habitant morbi tristique senectus. Nec sagittis aliquam malesuada bibendum arcu vitae. Quis risus sed vulputate odio ut enim blandit volutpat. Vitae nunc sed velit dignissim sodales. Duis convallis convallis tellus id interdum velit laoreet id. Faucibus ornare suspendisse sed nisi lacus sed viverra tellus in. Felis bibendum ut tristique et egestas quis. Ut etiam sit amet nisl purus in. Diam quam nulla porttitor massa id neque aliquam vestibulum. Cras semper auctor neque vitae tempus. Turpis in eu mi bibendum. Venenatis lectus magna fringilla urna porttitor rhoncus dolor purus. Cras fermentum odio eu feugiat. Sapien et ligula ullamcorper malesuada proin. Lorem sed risus ultricies tristique nulla aliquet. Accumsan tortor posuere ac ut. Vitae semper quis lectus nulla. Amet nulla facilisi morbi tempus iaculis. Ornare lectus sit amet est placerat. Libero justo laoreet sit amet cursus. Magna fermentum iaculis eu non diam phasellus. Ullamcorper malesuada proin libero nunc consequat interdum varius sit amet. Quam nulla porttitor massa id neque. Sed egestas egestas fringilla phasellus faucibus scelerisque eleifend. Quam vulputate dignissim suspendisse in est. Sit amet nisl purus in mollis nunc sed. At in tellus integer feugiat scelerisque varius morbi enim nunc. Malesuada proin libero nunc consequat. Cursus turpis massa tincidunt dui ut ornare. Proin sed libero enim sed. Id ornare arcu odio ut sem. Pellentesque nec nam aliquam sem et tortor consequat id. Bibendum arcu vitae elementum curabitur vitae nunc sed velit. Scelerisque eu ultrices vitae auctor eu augue ut lectus. Rhoncus mattis rhoncus urna neque viverra justo nec. Praesent tristique magna sit amet. Elit duis tristique sollicitudin nibh sit. In tellus integer feugiat scelerisque varius. Mi sit amet mauris commodo. Lectus urna duis convallis convallis. Fames ac turpis egestas sed tempus urna et. Donec ultrices tincidunt arcu non sodales neque sodales ut."

In [26]:
sentences = segment_sentences(lorem)
sentences.count("<s>")

500

#### Tokenizing the corpus

Clean and segment the corpus

In [27]:
clean_corpus = clean(corpus)

In [28]:
seg_corpus = segment_sentences(clean_corpus)

In [29]:
print(seg_corpus[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


In [30]:
print(corpus[-557:])

 ord till Klara Gulla. Hon hade fått större kärlek av sina föräldrar än någon annan han visste, och sådan kärlek måste vändas i välsignelse.

Då prästen sade detta, kom alla människor att se bort mot Klara Gulla, och de förundrade sig över vad de såg.

Prästens ord tycktes redan ha gått i uppfyllelse. Där stod Klara Fina Gulleborg ifrån Skrolycka, hon, som var uppkallad efter själva solen, vid sina föräldrars grav och lyste som en förklarad.

Hon var likaså vacker som den söndagen, då hon gick till kyrkan i den röda klänningen, om inte vackrare.








The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this:

```
<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s> 
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>
```

You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [31]:
words = re.findall('[^[\p{Z}\n]+', seg_corpus)
#words = seg_corpus.split()

In [32]:
print(len(words))

1041579


In [33]:
len(corpus)

5144417

In [34]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


The five last lines of the corpus should like this:

`['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']`



### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [35]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [36]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11624),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [37]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [38]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In [39]:
print(len(frequency_bigrams.keys()), "{:e}".format(len(words)**4), len(frequency_bigrams.keys()) / len(words)**2)

320129 1.176979e+24 2.9508055226853015e-07


In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus. This topic will be discussed during the lab session.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_. 

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [96]:
tot_words = len(words)

def unigram_lm(freq, swords):
    df = pd.DataFrame([], columns={'wi': str, 'C(wi)': int, '#words': int, 'P(wi)': float}).set_index('wi')
    wcounts = []
    wfreq = []
    
    for w in swords:
        wcounts.append(freq[w])
        wfreq.append(freq[w] / tot_words)
    
    for i, w in enumerate(swords):
        df = df.append({'wi': w, 'C(wi)': wcounts[i], '#words': tot_words, 'P(wi)': wfreq[i]}, ignore_index=True)
    
    metrics = {}
    metrics['tot_freq'] = 1
    
    for f in wfreq:
        metrics['tot_freq'] *= f
    
    metrics['geom_mean'] = metrics['tot_freq'] ** (1/len(wfreq))
    
    metrics['H'] = -math.log(metrics['tot_freq'], 2) / len(wfreq)
    metrics['pp'] = 2 ** metrics['H']
    
    print(df)
    print(f"Prob. unigrams: {metrics['tot_freq']}")
    print(f"Geometric mean prob.: {metrics['geom_mean']}")
    print(f"Entropy rate: {metrics['H']}")
    print(f"Perplexity: {metrics['pp']}")
          
    return metrics['pp']

In [97]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [98]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

   C(wi)   #words     P(wi)     wi
0  21108  1039623  0.020304    det
1  12090  1039623  0.011629    var
2  13514  1039623  0.012999     en
3   1332  1039623  0.001281   gång
4  13514  1039623  0.012999     en
5     16  1039623  0.000015   katt
6  16288  1039623  0.015667    som
7     97  1039623  0.000093  hette
8     87  1039623  0.000084   nils
9  58069  1039623  0.055856   </s>
Prob. unigrams: 5.375386888519094e-27
Geometric mean prob.: 0.0023607009389439992
Entropy rate: 8.726568996687329
Perplexity: 423.60300006799037


In [118]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [41]:
tot_words = len(words)

def bigram_lm(freq, freq_bi, swords):
    wcounts = []
    wcounts_bi = []
    wfreq = []
    
    for i in range(len(swords) - 1):
        w1, w2 = swords[i:i+2]
        wcounts.append(freq[w1])
        if (w1, w2) in freq_bi:
            wfreq.append(freq_bi[(w1, w2)] / freq[w1])
            wcounts_bi.append(freq_bi[(w1, w2)])
        else:
            wfreq.append(freq[w2] / tot_words)
            wcounts_bi.append(0)
    
    print("wi\t wi+1\t Ci,i+1\t C(i)\t P(wi)")
    for i in range(len(swords) - 1):
        if (swords[i], swords[i+1]) in freq_bi:
            print(f"{swords[i]}\t {swords[i+1]}\t {wcounts_bi[i]}\t {wcounts[i]}\t {wfreq[i]}")
        else:
            print(f"{swords[i]}\t {swords[i+1]}\t {wcounts_bi[i]}\t {wcounts[i]}\t 0\t *backoff: {wfreq[i]}")

    
    metrics = {}
    #metrics['tot_freq'] = freq[swords[0]] / tot_words
    metrics['tot_freq'] = 1

    for i in range(len(wfreq)):
        metrics['tot_freq'] *= wfreq[i]
    
    metrics['geom_mean'] = metrics['tot_freq'] ** (1/len(wfreq))
    
    metrics['H'] = -math.log(metrics['tot_freq'], 2) / len(wfreq)
    metrics['pp'] = 2 ** metrics['H']

    print(f"Prob. bigrams: {metrics['tot_freq']}")
    print(f"Geometric mean prob.: {metrics['geom_mean']}")
    print(f"Entropy rate: {metrics['H']}")
    print(f"Perplexity: {metrics['pp']}")
          
    return metrics['pp']

wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)


NameError: name 'swords' is not defined

In [101]:
sentence = '<s> det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
<s>	 det	 5568	 58069	 0.09588592880883087
det	 var	 3839	 21108	 0.1818741709304529
var	 en	 712	 12090	 0.058891645988420185
en	 gång	 706	 13514	 0.052242119283705785
gång	 en	 20	 1332	 0.015015015015015015
en	 katt	 6	 13514	 0.0004439840165754033
katt	 som	 2	 16	 0.125
som	 hette	 45	 16288	 0.002762770137524558
hette	 nils	 0	 97	 0	 *backoff: 8.368418166970142e-05
nils	 </s>	 2	 87	 0.022988505747126436
Prob. bigrams: 2.3763060554228703e-19
Geometric mean prob.: 0.013727461598244782
Entropy rate: 6.1867913143402475
Perplexity: 72.84667983539373


In [102]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

In [42]:
sentences = ["hej det var en fin dag", "varför är dagen så lång", "man lever som man lär", "hur kan fåglar flyga", "det var en fin natt i juli"]

for sentence in sentences:
    sent_words = f"<s> {sentence} </s>".split()
    perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
<s>	 hej	 1	 59047	 1.6935661422256845e-05
hej	 det	 0	 3	 0	 *backoff: 0.020265385534846612
det	 var	 3839	 21108	 0.1818741709304529
var	 en	 712	 12090	 0.058891645988420185
en	 fin	 11	 13514	 0.000813970697054906
fin	 dag	 0	 55	 0	 *backoff: 0.0009043961139769524
dag	 </s>	 150	 942	 0.1592356687898089
Prob. bigrams: 4.309129707332837e-16
Geometric mean prob.: 0.00638135338085044
Entropy rate: 7.2919218561612285
Perplexity: 156.70656995753637
wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
<s>	 varför	 184	 59047	 0.0031161617016952597
varför	 är	 7	 358	 0.019553072625698324
är	 dagen	 1	 6290	 0.0001589825119236884
dagen	 så	 2	 431	 0.004640371229698376
så	 lång	 21	 9149	 0.0022953328232593728
lång	 </s>	 6	 340	 0.01764705882352941
Prob. bigrams: 1.820769960172266e-15
Geometric mean prob.: 0.0034944259884605727
Entropy rate: 8.16072879375764
Perplexity: 286.17003287585396
wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
<s>	 man	 280	 59047	 0.004741985198231917
man	 lever	 1	 2322

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program it following the model of bigrams.

In [103]:
def trigrams(words):
    trigrams = [tuple(words[inx:inx + 3])
                for inx in range(len(words) - 2)]
    frequencies = {}
    for trigram in trigrams:
        if trigram in frequencies:
            frequencies[trigram] += 1
        else:
            frequencies[trigram] = 1
    return frequencies

In [ ]:
tot_words = len(words)

def trigram_lm(freq, freq_bi, freq_tri, swords):
    wcounts = []
    wcounts_tri = []
    wfreq = []
    
    for i in range(len(swords) - 1):
        w1, w2, w3 = swords[i:i+3]
        wcounts.append(freq[w1])
        
        if (w1, w2) in freq_bi:
            wfreq.append(freq_bi[(w1, w2)] / freq[w1])
            wcounts_bi.append(freq_bi[(w1, w2)])
        else:
            wfreq.append(freq[w2] / tot_words)
            wcounts_bi.append(0)
    
    print("wi\t wi+1\t Ci,i+1\t C(i)\t P(wi)")
    for i in range(len(swords) - 1):
        if (swords[i], swords[i+1]) in freq_bi:
            print(f"{swords[i]}\t {swords[i+1]}\t {wcounts_bi[i]}\t {wcounts[i]}\t {wfreq[i]}")
        else:
            print(f"{swords[i]}\t {swords[i+1]}\t {wcounts_bi[i]}\t {wcounts[i]}\t 0\t *backoff: {wfreq[i]}")
    
    metrics = {}
    #metrics['tot_freq'] = freq[swords[0]] / tot_words
    metrics['tot_freq'] = 1

    for i in range(len(wfreq)):
        metrics['tot_freq'] *= wfreq[i]
    
    metrics['geom_mean'] = metrics['tot_freq'] ** (1/len(wfreq))
    
    metrics['H'] = -math.log(metrics['tot_freq'], 2) / len(wfreq)
    metrics['pp'] = 2 ** metrics['H']

    print(f"Prob. bigrams: {metrics['tot_freq']}")
    print(f"Geometric mean prob.: {metrics['geom_mean']}")
    print(f"Entropy rate: {metrics['H']}")
    print(f"Perplexity: {metrics['pp']}")
          
    return metrics['pp']

In [104]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [105]:
starting_text = 'De'.lower()

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies.

In [106]:
cand_nbr = 5

In [107]:
candidates = []
for k, v in frequency_bigrams.items():
    if k[0].startswith(starting_text):
        candidates.append((k[0], v))

merged_candidates = defaultdict(int)
for k, f in candidates:
    merged_candidates[k] += f

sorted_candidates = {k: v for k, v in sorted(merged_candidates.items(), key=lambda item: item[1], reverse=True)}

current_word_predictions_1 = []
for i, k in enumerate(sorted_candidates.keys()):
    if i == cand_nbr:
        break
    current_word_predictions_1.append(k)

In [108]:
current_word_predictions_1

['det', 'de', 'den', 'dem', 'detta']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [109]:
current_text = "Det var en ".lower()

Tokenize this text and return a list of tokens. Call it `tokens`.

In [110]:
tokens = tokenize(current_text)

In [111]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`

In [112]:
candidates = []
for k, v in frequency_trigrams.items():
    if list(k[0:2]) == tokens[1:]:
        candidates.append((k[2], v))
    
    if list(k[1]) == tokens[-1]:
        candidates.append((k[1], v))
        
        
merged_candidates = defaultdict(int)
for k, f in candidates:
    merged_candidates[k] += f
    
sorted_candidates = {k: v for k, v in sorted(merged_candidates.items(), key=lambda item: item[1], reverse=True)}

next_word_predictions = []
for i, k in enumerate(sorted_candidates.keys()):
    if i == cand_nbr:
        break
    next_word_predictions.append(k)

In [113]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [114]:
current_text = "Det var en g".lower()

In [115]:
tokens = tokenize(current_text)
candidates = []
for k, v in frequency_trigrams.items():
    if list(k[0:2]) == tokens[1:3] and k[2].startswith(tokens[3]):
        candidates.append((k[2], bigram_lm(frequency, frequency_bigrams, tokens[:-1] + [k[2]])))
        
        
merged_candidates = {}
for k, f in candidates:
    merged_candidates[k] = f
    
sorted_candidates = {k: v for k, v in sorted(merged_candidates.items(), key=lambda item: item[1])}
print(sorted_candidates)
current_word_predictions_2 = []
for i, k in enumerate(sorted_candidates.keys()):
    if i == cand_nbr:
        break
    current_word_predictions_2.append(k)

wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
det	 var	 3839	 21108	 0.1818741709304529
var	 en	 712	 12090	 0.058891645988420185
en	 gång	 706	 13514	 0.052242119283705785
Prob. bigrams: 0.0005595585110215183
Geometric mean prob.: 0.08240403956289206
Entropy rate: 3.6011411276756427
Perplexity: 12.135327409972229
wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
det	 var	 3839	 21108	 0.1818741709304529
var	 en	 712	 12090	 0.058891645988420185
en	 grov	 5	 13514	 0.00036998668047950276
Prob. bigrams: 3.962878973240216e-06
Geometric mean prob.: 0.015824752865310397
Entropy rate: 5.981673220470249
Perplexity: 63.192140092886426
wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
det	 var	 3839	 21108	 0.1818741709304529
var	 en	 712	 12090	 0.058891645988420185
en	 gås	 11	 13514	 0.000813970697054906
Prob. bigrams: 8.718333741128473e-06
Geometric mean prob.: 0.020581538225159682
Entropy rate: 5.602505379220271
Perplexity: 48.58723332824373
wi	 wi+1	 Ci,i+1	 C(i)	 P(wi)
det	 var	 3839	 21108	 0.1818741709304529
var	 en	 712	 12090	 0.05889164

In [116]:
current_word_predictions_2

['gång', 'gammal', 'god', 'gård', 'glädje']

## Checked answers

The system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [119]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'dem', 'detta'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gång', 'gammal', 'god', 'gård', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [121]:
STIL_ID = ["ol7501en-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [122]:
ANSWER = str((perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2))
ANSWER

"(423, 72, ['det', 'de', 'den', 'dem', 'detta'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gång', 'gammal', 'god', 'gård', 'glädje'])"

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [123]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [124]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [125]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'incorrect',
 'signature': None,
 'submission_id': 'fad61192-4634-467d-94d9-61ea8d252acc'}

## Reading

<p>As an application of n-grams, execute the Jupyter notebook by Peter Norvig <a
        href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">
    here</a>. Just run all the cells and be sure that you understand the code.
    You will find the data <a href="http://norvig.com/ngrams/">here</a>.</p>
<p>In your report, you will also describe one experiment with a long string of words
    your will create yourself or copy from a text you like. You will remove all the punctuation and
    white spaces from this string. Set this string in lowercase letters.</p>
<p>You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and
    run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. </p>
<p>You will comment the segmentation results you obtain with unigram and bigram models.
</p>